In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import os

particle_id = [9] # identity of the particle starts from 1

In [ ]:
# To load the initial particle data time=0
XY0 = pd.read_csv(f"ini_state_q1.txt", header=None, delimiter='\t', names=['x','y','rad','theta','speed','type'])

In [ ]:
for pid in particle_id:
    trajectory_df = pd.read_csv(f"trajectory_{pid}.txt", header=0, delimiter='\t', names = ['time','x','y'])
    # Converting to DataFrame
    x = trajectory_df['x']
    y = trajectory_df['y']
    TIME = np.array(trajectory_df['time'])

    # Computing motion vectors
    dx = np.diff(x)
    dy = np.diff(y)
    x_mid = x[:-1]
    y_mid = y[:-1] # No velocity for the last (x,y)
    time_mid = TIME[:-1]  # Match shape with dx, dy for coloring

    # Normalize and map to colors - Use time_mid instead of TIME
    norm = mcolors.Normalize(vmin=TIME.min(), vmax=TIME.max())
    cmap = plt.cm.plasma #viridis, plasma, inferno, magma, cividis
    colors = cmap(norm(time_mid))  # Use time_mid to match quiver array dimensions

    # Creating figure and axes
    fig, ax = plt.subplots(figsize=(7, 7))

    # Plotting quiver with colored arrows on the axes
    q = ax.quiver(x_mid, y_mid, dx, dy, angles='xy', scale_units='xy', scale=1, 
                  color=colors, width=0.007)

    # Setting axes to have equal scaling
    ax.set_aspect('equal', adjustable='box')

    # Adding a colorbar tied to the axes
    sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])  # Required for colorbar
    cbar = fig.colorbar(sm, ax=ax, label='Time', shrink=0.8)

    # Adding the start and stop positions for the particle
    start_color = cmap(norm(TIME[0]))
    end_color = cmap(norm(TIME[-1]))
    ax.plot(x_mid.iloc[0], y_mid.iloc[0], 'o', color=start_color, markersize=8, label='Start')
    ax.plot(x_mid.iloc[-1], y_mid.iloc[-1], 'o', color=end_color, markersize=8, label='End')

    # Adding legend for start/end points
    ax.legend()

    # Filtering boundary particles
    df_boundaries = XY0[XY0['type'] == 0]

    # Drawing each circle
    for _, row in df_boundaries.iterrows():
        circle = patches.Circle((row['x'], row['y']), radius=row['rad'], edgecolor='black', 
            facecolor='none', linewidth=0.75)
        ax.add_patch(circle)

    ax.set_xlim(-2,32)
    ax.set_ylim(-2,32)

    # Adding grid and labels
    # ax.grid(True, alpha=0.3)
    ax.set_xlabel('X Position')
    ax.set_ylabel('Y Position')
    ax.set_title(f'Particle {pid} Trajectory with Time-Colored Vectors')


    # Saving the figure
    plt.savefig(f"quiver_plot_traj_particle_{pid}.png", dpi=1000, bbox_inches='tight')

    plt.show()